<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DiracEq_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Define constants
m = 9.11e-31   # mass of electron (kg)
c = 3e8        # speed of light (m/s)
e = 1.6e-19    # elementary charge (C)
hbar = 1.05e-34   # Planck's constant / 2pi (J s)
L = 1e-8       # length of plasma (m)
B0 = 1         # magnetic field strength (T)
Vo = 10        # amplitude of potential barrier (eV)
S0 = L/2       # center of potential barrier (m)
Nz = 1001      # number of spatial grid points
z = np.linspace(0, L, Nz)
dz = z[1] - z[0]
dt = 1e-18     # time step size (s)
omega = e * B0 / (m * c**2)   # angular frequency of Larmor motion (1/s)

# Define potential energy function
def U(q, S, t):
    return Vo * np.exp(-((z - S - q * omega * t)**2) / (2 * (L/10)**2))

# Define initial conditions for Dirac field and new field chi
psi = np.zeros(Nz, dtype=np.complex128)
chi = np.zeros(Nz, dtype=np.complex128)
psi[0] = 1 + 0j
q = np.zeros(Nz)
S = np.zeros(Nz)
for j in range(Nz):
    q[j] = j - (Nz - 1) / 2
    S[j] = S0

# Define ode function for solving coupled equationsdef ode_func(y, z):
def ode_func(y, z):
    psi = y[0]
    chi = y[1]
    
    dpsi_dz = (psi - np.roll(psi, 1)) / dz
    dpsi_dz[0] = 0
    dchi_dz = (chi - np.roll(chi, 1)) / dz
    dchi_dz[0] = 0.
    
    dpsi_dt = -1j * (m * c**2 / hbar) * A * chi
    dchi_dt = -1j * (m * c**2 / hbar) * A * psi - (q / hbar) * (U_j - m * c**2 * (q * e * B0 / (m * c**2)) + dU_dq) * chi
    
    return np.array([dpsi_dz, dchi_dz, dpsi_dt, dchi_dt])


# Define spatial grid
Nz = 500
z = np.linspace(0, L, Nz)

# Initialize psi and chi arrays
psi = np.zeros(Nz, dtype=np.complex128)
chi = np.zeros(Nz, dtype=np.complex128)
psi[0] = 1 + 0j

# Define initial conditions for chi and psi
chi_prev = np.zeros(Nz, dtype=np.complex128)
chi_prev[0] = 1 + 0j

psi_prev = np.zeros(Nz, dtype=np.complex128)
psi_prev[0] = 1 + 0j

# Solve for wavefunctions
wavefunctions = np.zeros((Nz, 5), dtype=np.complex128)
energy_levels = []
for n in range(5):
    # Define initial conditions for Dirac field and new field chi
    psi = np.zeros(Nz, dtype=np.complex128)
    chi = np.zeros(Nz, dtype=np.complex128)
    psi[0] = 1 + 0j

    # Solve for wavefunction using odeint
    y0 = [psi[0], chi[0]]
    sol = odeint(ode_func, y0, z)

    # Store wavefunction for this energy level
    wavefunctions[:, n] = sol[:, 0]

    # Normalize wavefunction
    norm = np.sqrt(np.trapz(np.abs(wavefunctions[:, n])**2, z))
    wavefunctions[:, n] = wavefunctions[:, n] / norm

    # Calculate energy eigenvalue for this wavefunction
    E = (n + 1/2) * hbar * omega
    energy_levels.append(E)

# Plot energy levels
plt.plot(np.arange(1, len(energy_levels) + 1), energy_levels, 'bo')
plt.xlabel('Energy Level')
plt.ylabel('Energy (J)')
plt.show()

# Plot wavefunctions
for n in range(5):
    plt.plot(z, np.abs(wavefunctions[:, n])**2)
plt.xlabel('Position (m)')
plt.ylabel('Probability Density')
plt.show()


<ipython-input-18-aa526bc2cea7>:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([dpsi_dz, dchi_dz, dpsi_dt, dchi_dt])


TypeError: ignored